In [30]:
import pandas as pd
import requests
import os, sys
from fuzzywuzzy import fuzz
import dedupe
import numpy as np

In [3]:
#Get TV Stations by DMA
dma = 'PHOENIX (PRESCOTT)'
url = "https://publicfiles.fcc.gov/api/service/facility/search/"+dma.split()[0]+".json"
r = requests.get(url).json()
tv = pd.DataFrame(r['results']['globalSearchResults']['tvFacilityList'])
tv = tv[tv.nielsenDma==dma]
tv = tv[['callSign','communityCity','communityState','networkAfil','nielsenDma','virtualChannel','id']]

In [ ]:
# Create folders for each station.
def createStationFolders(s):
    path = "/media/andrew/F08C9B848C9B444E/analysis/tv/buys/"+s
    try: 
        os.mkdir(path, 755 )
    except FileExistsError:
        print(path+" exists")
    try:
        os.mkdir(path+"/Political Files", 755)
    except FileExistsError:
        print('')
    try:
        os.mkdir(path+"/Political Files/2018", 755)
    except FileExistsError:
        print('')
tv['callSign'].apply(createStationFolders)

In [4]:
#Get the head political Folder
cycle = '14'
def getFileFolder(iD):
    url = 'https://publicfiles.fcc.gov/api/manager/folder/path.json'
    folder = 'Political Files/20'+cycle
    r = requests.get(url, params={"folderPath":folder, "entityId": iD, 'sourceService':'tv'}).json()
    if r['statusCode'] == 200:
        return r['folder'][0]['entity_folder_id']
    else:
        return None
tv[(cycle+'PoliticalFolder')] = tv['id'].apply(getFileFolder)

In [ ]:
# Create all directory folders
cycle = '14'
import time
def createBuyFolders(s):
    print(s)
    if s[(cycle+'PoliticalFolder')] != None:
        url = "https://publicfiles.fcc.gov/api/manager/folder/id/"+s[(cycle+'PoliticalFolder')]+".json"
        digToNextLevel(s[(cycle+'PoliticalFolder')], s['id'], s['callSign'])
def digToNextLevel(fId, tvId, sCs):
    path = "/media/andrew/F08C9B848C9B444E/analysis/tv/buys/"
    url = "https://publicfiles.fcc.gov/api/manager/folder/id/"+fId+".json"
    r = requests.get(url,params={'entityId':tvId}).json()
    if r['statusCode'] == 200:
        try:
            os.mkdir(path+sCs+"/"+r['folder']['folder_path'], 755)
        except FileExistsError:
            print(path+sCs+"/"+r['folder']['folder_path']+" exists")
        if len(r['folder']['subfolders']) > 0:
            for sf in r['folder']['subfolders']:
                digToNextLevel(sf["entity_folder_id"], sf["entity_id"], sCs)
    else:
        print(r['statusMessage'])
        time.sleep(10)
        digToNextLevel(fId, tvId, sCs)
tv[[(cycle+'PoliticalFolder'),'id','callSign']].apply(createBuyFolders, axis=1)

In [108]:
stations = [];
for f in list(os.scandir("/media/andrew/F08C9B848C9B444E/analysis/tv/buys/")):
    f.is_dir()

In [3]:
pacs = pd.read_csv('pacs.csv')

In [62]:
juris = ['Federal', 'Local', 'Non-Candidate Issue Ads', 'State']
fedrace = ['President', 'US Senate', 'US House']
pacs = []
def getAllPacs(s):
    basepath = "/media/andrew/F08C9B848C9B444E/analysis/tv/buys/"+s['callSign']+'/Political Files/'+cycle+'/'
    try:
        os.scandir(basepath)
        for j in juris:
            path = basepath + j
            if j == 'Federal':
                for r in fedrace:
                    fedpath = path+'/'+r
                    if r == 'US House': # Sometimes House folders have districts
                        dist = []
                        fcontents = list(os.scandir(fedpath))
                        if len(fcontents) >0:
                            if fcontents[0].is_dir():
                                for f in fcontents:
                                    distpath = fedpath+'/'+f.name
                                    for p in list(os.scandir(distpath)):
                                        pacs.append({"Name": p.name, "Jurisdiction": j, "Race": None, "Station":s['callSign'], 
                                                     "Path": p.path, 'District': f.name, "Cycle": cycle})
                        else:
                            for f in fcontents:
                                pacs.append({"Name": p.name, "Jurisdiction": j, "Race": None, "Station":s['callSign'], "Path": p.path, 'District': None, "Cycle": cycle})
                    else:
                        for p in list(os.scandir(fedpath)):
                            if p.is_dir():
                                pacs.append({"Name": p.name, "Jurisdiction": j, "Race": None, "Station":s['callSign'], "Path": p.path, 'District': None, "Cycle": cycle})
            if j == 'State':
                fcontents = list(os.scandir(path))
                if len(fcontents)>0:
                    firstpath = path+'/'+fcontents[0].name
                    firstcontents = list(os.scandir(firstpath))
                    if len(firstcontents)>0:
                        if firstcontents[0].is_dir():
                            for f in fcontents:
                                for p in firstcontents:
                                    pacs.append({"Name": p.name, "Jurisdiction": j, "Race": f.name, "Station":s['callSign'], "Path": p.path, 'District': None,
                                                 "Cycle": cycle})
                        else:
                            for p in list(os.scandir(path)):
                                if p.is_dir():
                                    pacs.append({"Name": p.name, "Jurisdiction": j, "Race": None, "Station":s['callSign'], 
                                                 "Path": p.path, 'District': None, "Cycle": cycle})
                    else:
                        for p in list(os.scandir(path)):
                            if p.is_dir():
                                pacs.append({"Name": p.name, "Jurisdiction": j, "Race": None, "Station":s['callSign'], 
                                             "Path": p.path, 'District': None, "Cycle": cycle})
            if (j != 'Federal') & (j != 'State'):
                for p in list(os.scandir(path)):
                    if p.is_dir():
                        pacs.append({"Name": p.name, "Jurisdiction": j, "Race": None, "Station":s['callSign'], "Path": p.path, 'District': None, "Cycle": cycle})
    except:
        return

cycles = ['2018','2016','2014']

for c in cycles:
    cycle = c
    tv.apply(getAllPacs, axis=1)
pacs = pd.DataFrame(pacs)

test
test
test
test
test


In [5]:
pd.set_option('display.max_rows',500)
# Transform words
pacs['Name'] = pacs.Name.str.replace('AZ ','ARIZONA ', case=False)
pacs['Name'] = pacs.Name.str.replace(' AZ',' ARIZONA', case=False)
pacs['Name'] = pacs.Name.str.replace(' Prty ',' Party ', case=False)
pacs['Name'] = pacs.Name.str.replace(' ATT ',' ATTORNEY ', case=False)
pacs['Name'] = pacs.Name.str.replace(' ATY ',' ATTORNEY ', case=False)
pacs['Name'] = pacs.Name.str.replace(' SEC ',' SECRETARY ', case=False)
pacs['Name'] = pacs.Name.str.replace(' GEN',' GENERAL', case=False)
pacs['Name'] = pacs.Name.str.replace(' GEN ',' GENERAL ', case=False)
pacs['Name'] = pacs.Name.str.replace(' SOS ',' SECRETARY OF STATE ', case=False)
pacs['Name'] = pacs.Name.str.replace(' AG ',' ATTORNEY GENERAL ', case=False)
pacs['Name'] = pacs.Name.str.replace(' AG',' ATTORNEY GENERAL', case=False)
pacs['Name'] = pacs.Name.str.replace(' DEM ',' DEMOCRATIC ', case=False)
pacs['Name'] = pacs.Name.str.replace(' REP ',' REPUBLICAN ', case=False)
pacs['Name'] = pacs.Name.str.replace(' DEMS ',' DEMOCRATIC PARTY ', case=False)
pacs['Name'] = pacs.Name.str.replace(' GOV ', 'GOVERNOR', case=False)
pacs['Name'] = pacs.Name.str.replace('AZDP',' ARIZONA DEMOCRATIC PARTY ', case=False)
pacs['Name'] = pacs.Name.str.replace('AZRP ',' ARIZONA REPUBLICAN PARTY ', case=False)
pacs['Name'] = pacs.Name.str.replace(' AZDEMS ',' ARIZONA DEMOCRATIC PARTY ', case=False)


pacs['Name'] = pacs.Name.str.replace('for Arizona', '', case=False)


#Strip
pacs['Name'] = pacs.Name.str.replace(' - AZGOP','', case=False)
pacs['Name'] = pacs.Name.str.replace(' GOP','', case=False)
pacs['Name'] = pacs.Name.str.replace('GOP','', case=False)
pacs['Name'] = pacs.Name.str.replace('-','', case=False)
pacs['Name'] = pacs.Name.str.replace(' PAC', '', case=False)
pacs['Name'] = pacs.Name.str.replace('2018',' ', case=False)
pacs['Name'] = pacs.Name.str.replace('- General Election','', case=False)
pacs['Name'] = pacs.Name.str.replace('- Primary Election','', case=False)
pacs['Name'] = pacs.Name.str.replace(' IE','', case=False)

# Race Names
pacs['Name'] = pacs.Name.str.replace(' FOR Governor', '', case=False)
pacs['Name'] = pacs.Name.str.replace(' FOR SECRETARY OF STATE', '',case=False)
pacs['Name'] = pacs.Name.str.replace(' FOR ATTORNEY GENERAL','', case=False)
pacs['Name'] = pacs.Name.str.replace(' FOR GENERAL','', case=False)
pacs['Name'] = pacs.Name.str.replace(' GENERAL ELECTION','', case=False)
pacs['Name'] = pacs.Name.str.replace('- GOP','', case=False)
pacs['Name'] = pacs.Name.str.replace(' IE','', case=False)
pacs['Name'] = pacs.Name.str.replace(' FOR','', case=False)
pacs['Name'] = pacs.Name.str.replace(' FOR ARIZONA','', case=False)
pacs['Name'] = pacs.Name.str.upper()
#pacs = pacs.groupby('Name', as_index=False).count()
pacs[['Name','Station']].sort_values(by="Name")[0:1000]

,Name,Station
344,ARIZONA DEMOCRATIC PARTY JANUARY CONTREAS AT...,KPNX
6,45 COMMITTE,KASW
98,45 COMMITTEE,KPHO-TV
37,45 COMMITTEE,KNXV-TV
142,45 COMMITTEE,KSAZ-TV
256,45 COMMITTEE,KUTP
317,45 COMMITTEE,KPNX
557,60 PLUS ASSOCIATION,KTVK
503,60 PLUS ASSOCIATION,KPHO-TV
184,ACLU,KTAZ


In [10]:
pacs[['Name','Jurisdiction']].sort_values(by="Name")[0:1000]

,Name,Jurisdiction
138,45 COMMITTEE,Non-Candidate Issue Ads
6,45 Committe PAC,Non-Candidate Issue Ads
45,45 Committee,Non-Candidate Issue Ads
392,45 Committee,Non-Candidate Issue Ads
210,45 Committee,Non-Candidate Issue Ads
481,45 Committee 2018,Non-Candidate Issue Ads
139,ACLU,Non-Candidate Issue Ads
278,ACLU,Non-Candidate Issue Ads
482,ACLU 2018,Non-Candidate Issue Ads
46,ACLU AZ,Non-Candidate Issue Ads


In [ ]:
pacs[['Name','Station']].sort_values(by="Name")[0:1000]

In [260]:
from fuzzywuzzy import fuzz
fuzz.token_sort_ratio('T OHALLERAN AZ-1', 'TIFFANY SHEDD AZ-1')

53

In [263]:
tv.head()

,callSign,communityCity,communityState,networkAfil,nielsenDma,virtualChannel,id,18PoliticalFolder,16PoliticalFolder
0,KAET,PHOENIX,AZ,PBS,PHOENIX (PRESCOTT),8,2728,a195b284-3823-9311-3fd0-ab1eb7945dce,a195b284-3823-9311-3fd0-ab1eb7945dce
1,KASW,PHOENIX,AZ,CW,PHOENIX (PRESCOTT),61,7143,3dd55d88-d62c-0cff-eeb8-822f84cdc06e,3dd55d88-d62c-0cff-eeb8-822f84cdc06e
4,KNXV-TV,PHOENIX,AZ,ABC,PHOENIX (PRESCOTT),15,59440,f532836f-2248-72e2-1974-cca90b96998d,f532836f-2248-72e2-1974-cca90b96998d
5,KPAZ-TV,PHOENIX,AZ,TRINITY BROADCASTING NETWORK,PHOENIX (PRESCOTT),21,67868,872c413b-96b9-549e-e6bc-1710e893d9bf,872c413b-96b9-549e-e6bc-1710e893d9bf
7,KPHO-TV,PHOENIX,AZ,CBS,PHOENIX (PRESCOTT),5,41223,1d45d3b6-edbf-15ce-4113-adc9d0aae463,1d45d3b6-edbf-15ce-4113-adc9d0aae463


In [63]:
pacs.Race.unique()

array([None, 'KTVW  David Garcia for AZ GOV of AZ',
       'KTVW AZ Education Assoc-Teacher Walkout',
       'KTVW Doug Ducey for GOV of AZ  2018', 'KTVW- Sears for AZ',
       'Women Vote', 'AZ Congress', 'Attorney General', 'Governor',
       'Secretary of State', 'Superindintendent of Public Education',
       'ATTORNEY GENERAL', 'AZ SUPERINTENDENT', 'SECRETARY OF STATE',
       'Arizona'], dtype=object)

In [190]:
p = pd.read_csv('pacs.csv')
p.head()

,Cycle,District,Jurisdiction,Name,Path,Race,Station
0,2018,NaN,Federal,Kyrsten Sinema,/media/andrew/F08C9B848C9B444E/analysis/tv/buy...,NaN,KASW
1,2018,NaN,Federal,Kyrsten Sinema DSCC,/media/andrew/F08C9B848C9B444E/analysis/tv/buy...,NaN,KASW
2,2018,NaN,Federal,McSally for Senate,/media/andrew/F08C9B848C9B444E/analysis/tv/buy...,NaN,KASW
3,2018,AZ 01,Federal,DCCC T OHALLERAN AZ-1,/media/andrew/F08C9B848C9B444E/analysis/tv/buy...,NaN,KASW
4,2018,AZ 01,Federal,OHALLERAN AZ-1,/media/andrew/F08C9B848C9B444E/analysis/tv/buy...,NaN,KASW


In [202]:
# Get info from name.
p['NameSimpl'] = p['Name']
p['Party'] = ""

# Transform words
p['NameSimpl'] = p['NameSimpl'].str.replace('AZDP',' DEMOCRATIC PARTY ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('AZRP',' REPUBLICAN PARTY ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('AZDEMS',' DEMOCRATIC PARTY ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('AZ ','ARIZONA ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' AZ',' ARIZONA', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' Prty ',' Party ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' ATT ',' ATTORNEY ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' ATY ',' ATTORNEY ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' SEC ',' SECRETARY ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' GEN ',' GENERAL ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' SOS ',' SECRETARY OF STATE ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' SOS',' SECRETARY OF STATE ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' AG ',' ATTORNEY GENERAL ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' AG',' ATTORNEY GENERAL', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' DEM ',' DEMOCRATIC ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' REP ',' REPUBLICAN ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' DEMS ',' DEMOCRATIC PARTY ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' GOV ', ' GOVERNOR ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' PHX ', ' PHOENIX ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('National Republican Senatorial Committee','NRSC', case=False)



#PAC
p.loc[p['Jurisdiction'] == 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace(' 4 ', ' For ', case=False)
p.loc[p['Jurisdiction'] == 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace(' ASSC ', ' Association ', case=False)

#Candidate
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('For', '', case=False)
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 
      'NameSimpl'] = p['NameSimpl'].str.replace('- General Election', '', case=False)
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 
      'NameSimpl'] = p['NameSimpl'].str.replace('- Primary Election', '', case=False)
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 
      'NameSimpl'] = p['NameSimpl'].str.replace('Phoenix', '', case=False)
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 
      'NameSimpl'] = p['NameSimpl'].str.replace('Arizona', '', case=False)

#Party
p.loc[p['Name'].str.contains(' R ', case=False), 'Party'] = "Republican"
p.loc[p['Name'].str.contains(' D ', case=False), 'Party'] = "Democrat"
p.loc[p['Name'].str.contains('Republican', case=False), 'Party'] = "Republican"
p.loc[p['Name'].str.contains('Democrat', case=False), 'Party'] = "Democrat"
p.loc[(p['Jurisdiction'] != 'Non-Candidate Issue Ads') & 
      (p['NameSimpl'].str.contains('Republican Party', case=False)==False),
      'NameSimpl'] = p['NameSimpl'].str.replace(' Republican ', '', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' Democrat ','', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' D ','', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace(' R ','', case=False)

#Federal
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace(' US ', '', case=False)
#Senate
p.loc[p['Name'].str.contains('Senate', case=False), 'Race'] = "Senate"
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('Senate', '', case=False)
p.loc[p['Name'].str.contains('DSCC', case=False), 'Race'] = "Senate"
p.loc[p['Name'].str.contains('DSCC', case=False), 'Party'] = "Democrat"
p.loc[p['Name'].str.contains('NRSC', case=False), 'Race'] = "Senate"
p.loc[p['Name'].str.contains('NRSC', case=False), 'Party'] = "Republican"
# House
p.loc[p['Name'].str.contains('House', case=False), 'Race'] = "House"
p.loc[p['Name'].str.contains('Congress', case=False), 'Race'] = "House"
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('House', '', case=False)
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('Congress', '', case=False)
p.loc[p['Name'].str.contains('DCCC', case=False), 'Race'] = "House"
p.loc[p['Name'].str.contains('DCCC', case=False), 'Party'] = "Democrat"
p.loc[p['Name'].str.contains('NRCC', case=False), 'Race'] = "House"
p.loc[p['Name'].str.contains('NRCC', case=False), 'Party'] = "Republican"

#State
p.loc[p['NameSimpl'].str.contains('Arizona Democratic Party', case=False), 'Party'] = "Democrat"
p.loc[p['NameSimpl'].str.contains('Arizona Republican Party', case=False), 'Party'] = "Republican"
#Governor
p.loc[p['NameSimpl'].str.contains('RGA', case=False), 'Party'] = "Republican"
p.loc[p['NameSimpl'].str.contains('DGA', case=False), 'Race'] = "Governor"
p.loc[p['NameSimpl'].str.contains('RGA', case=False), 'Race'] = "Governor"
p.loc[p['NameSimpl'].str.contains('Governor', case=False), 'Race'] = "Governor"
p.loc[p['NameSimpl'].str.contains('DGA', case=False), 'Party'] = "Democrat"
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('Governor', '', case=False)
#AG
p.loc[p['NameSimpl'].str.contains('Attorney General', case=False), 'Race'] = "Attorney General"
p.loc[p['Jurisdiction'] != 'Non-Candidate Issue Ads', 'NameSimpl'] = p['NameSimpl'].str.replace('Attorney General', '', case=False)
p.loc[p['NameSimpl'].str.contains('RAGA', case=False), 'Party'] = "Republican"
p.loc[p['NameSimpl'].str.contains('DAGA', case=False), 'Race'] = "Attorney General"
p.loc[p['NameSimpl'].str.contains('RAGA', case=False), 'Race'] = "Attorney General"
p.loc[p['NameSimpl'].str.contains('DAGA', case=False), 'Party'] = "Democrat"
#SOS
p.loc[p['NameSimpl'].str.contains('Secretary of State', case=False), 'Race'] = "Secretary of State"
p['NameSimpl'] = p['NameSimpl'].str.replace('Secretary of State', '', case=False)

#Mayor
p.loc[p['NameSimpl'].str.contains('Mayor', case=False), 'Race'] = "Mayor"
p['NameSimpl'] = p['NameSimpl'].str.replace('Mayor', '', case=False)


#All
p['NameSimpl'] = p['NameSimpl'].str.replace('-',' ', case=False)
p['NameSimpl'] = p['NameSimpl'].str.replace('2018', '')
p['NameSimpl'] = p['NameSimpl'].str.replace('2016', '')
p['NameSimpl'] = p['NameSimpl'].str.replace('2014', '')

#Party Committees
p['NameSimpl'] = p['NameSimpl'].str.replace('Democratic Party', '')
p['NameSimpl'] = p['NameSimpl'].str.replace('Republican Party', '')
p['NameSimpl'] = p['NameSimpl'].str.replace('Behalf of', '', case=False)

p[0:1000][['NameSimpl', 'Name', 'Jurisdiction', 'Party', 'Race']].sort_values(by="NameSimpl")

p['Cycle'] = p['Cycle'].astype(str)
p = p.replace({np.nan:None})

In [196]:
fuzz.token_set_ratio('Katie Hobbs', 'Democratic Party Hobbs')

62

In [211]:
matched = []
curName = ""

def matcher(r):
    matcher.race = None
    matcher.district = None
    matcher.party = None
    def closestNames(n):
        matchVal = fuzz.token_sort_ratio(n.NameSimpl, r.NameSimpl)
        matchVal2 = fuzz.token_set_ratio(n.NameSimpl, r.NameSimpl)
        if matchVal>70:
            print(matchVal," ", matchVal2)
            print(n)
            if (n.Race != None) & (n.Race != "") & (matcher.race != None):
                matcher.race = n.Race
            if (n.District != None) & (n.District != "") & (matcher.district != None):
                matcher.district = n.District
            if (n.Party != None) & (n.Party != "") & (matcher.party != None):
                matcher.party = n.Party
            return True
        else:
            return False
    possmatch = p[(p.Cycle==r.Cycle)&(p.Jurisdiction==r.Jurisdiction)]
    curName = r.NameSimpl
    match = possmatch[possmatch.apply(closestNames, axis=1)]
    
    if (matcher.race != None) & (r['Race'] != None):
        r['Race'] = matcher.race
    if (matcher.district != None) & (r['District'] != None):
        r['District'] = matcher.district
    if (matcher.party != None) & (r['Party'] != None):
        r['Party'] = matcher.party
    return r
pz = p[171:172].apply(matcher, axis=1)

78   83
Cycle                                                        2018
District                                                     None
Jurisdiction                                                State
Name                           Katie Hobbs for Secretary of State
Path            /media/andrew/F08C9B848C9B444E/analysis/tv/buy...
Race                                           Secretary of State
Station                                                      KASW
NameSimpl                                           Katie Hobbs  
Party                                                    Democrat
Name: 28, dtype: object
83   100
Cycle                                                        2018
District                                                     None
Jurisdiction                                                State
Name            Arizona Democratic Party - Hobbs for AZ Secret...
Path            /media/andrew/F08C9B848C9B444E/analysis/tv/buy...
Race                               

In [210]:
pz

,Cycle,District,Jurisdiction,Name,Path,Race,Station,NameSimpl,Party
171,2018,None,State,AZ Democratic Prty Behalf of K Hobbs,/media/andrew/F08C9B848C9B444E/analysis/tv/buy...,None,KSAZ-TV,K Hobbs,Democrat
